# Appendix B

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score  
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import roc_curve 
from sklearn.metrics import classification_report
from sklearn import metrics


%matplotlib inline
sns.set_context('notebook') 

# Load Dataset (987 genes associated with Gemcitabine Response)

In [2]:
data = pd.read_csv("900 amp and del from fisher exact.csv")
print(data.shape)
# data.head()

(57, 988)


In [3]:
seed = 0

In [4]:
threshold = 0.47

In [5]:
from sklearn.model_selection import train_test_split

# try to use only important quantitative columns + binary columns
## define x_topf and y_topf
X= data.iloc[:,1:].values
print(type(X))
print(X.shape)

y = data.iloc[:,0].values
print(type(y))
print(y)


# create x_train and y_label
X_train,X_val,y_train,y_val = train_test_split(X, y,train_size=0.8, random_state=0,stratify=y)


print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

<class 'numpy.ndarray'>
(57, 987)
<class 'numpy.ndarray'>
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(45, 987)
(12, 987)
(45,)
(12,)


# L2 Regularised Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

# Logistic regression
logit_l2 = LogisticRegressionCV(Cs=10, class_weight='balanced', cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)
logit_l2.fit(X_train, y_train)
print(logit)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


In [15]:
for mylogitmodel in [logit_l2]:
    print("=======================")
    print(mylogitmodel)
    print("--------------")
    y_prob_lg = mylogitmodel.predict_proba(X_val)
    y_pred_lg =(y_prob_lg[:,1]>0.45).astype(int)
    print( confusion_matrix(y_val, y_pred_lg) )
    print(classification_report(y_val, y_pred_lg))
    auc_lg=roc_auc_score(y_val, y_prob_lg[:,1]).round(3)
    print("auc = {}".format(auc_lg))
    print("f1 score = {}".format(f1_score(y_val, y_pred_lg)))
    print(y_prob_lg)
    print(y_pred_lg)
    print(y_val)

LogisticRegressionCV(Cs=10, class_weight='balanced', cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)
--------------
[[4 3]
 [1 4]]
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.57      0.80      0.67         5

    accuracy                           0.67        12
   macro avg       0.69      0.69      0.67        12
weighted avg       0.70      0.67      0.67        12

auc = 0.771
f1 score = 0.6666666666666666
[[0.67144102 0.32855898]
 [0.62656911 0.37343089]
 [0.5102885  0.4897115 ]
 [0.5102885  0.4897115 ]
 [0.51078937 0.48921063]
 [0.54220644 0.45779356]
 [0.5147606  0.4852394 ]
 [0.52713865 0.47286135]
 [0.5102885  0.4897115 ]
 [0.71516456

In [16]:
numiterations = 10
threshold = 0.50
listf1scores = []
listprecision = []
listrecall = []
listaccuracy = []
listauc = []
listrownames = []
nsamples = len(X)
totaltosample = int(0.2*(nsamples))
df_final = data
for i in range(numiterations):
    listrownames.append(i+1)
    df_boot = df_final.iloc[np.random.randint(nsamples, size=totaltosample),:]
    X_boot = df_boot.iloc[:,1:].values
    y_val_final = df_boot.iloc[:,0].values
    y_pred_prob = logit_l2.predict_proba(X_boot)
    y_pred =(y_pred_prob[:,1]>threshold).astype(int)
    listf1scores.append(f1_score(y_val_final, y_pred).round(4))
    listaccuracy.append(accuracy_score(y_val_final, y_pred).round(4))
    listprecision.append(precision_score(y_val_final, y_pred).round(4))
    listrecall.append(recall_score(y_val_final, y_pred).round(4))
    listauc.append(roc_auc_score(y_val_final, y_pred_prob[:,1]).round(4))
    
listf1scores.append(np.mean(listf1scores).round(4))
listaccuracy.append(np.mean(listaccuracy).round(4))
listprecision.append(np.mean(listprecision).round(4))
listrecall.append(np.mean(listrecall).round(4))
listauc.append(np.mean(listauc).round(4))
listrownames.append("Average")
df_results = pd.DataFrame(columns=["Iteration","F1 Score","Accuracy","Precision","Recall"])
df_results["Iteration"] = listrownames # [1,2,3,4,5,6,7,8,9,10,"Average"]
df_results["F1 Score"] = listf1scores
df_results["Accuracy"] = listaccuracy
df_results["Precision"] = listprecision
df_results["Recall"] = listrecall
df_results["AUC"] = listauc

# display the results table
df_results

,Iteration,F1 Score,Accuracy,Precision,Recall,AUC
0,1,0.5714,0.7273,1.0000,0.4000,0.9500
1,2,0.6667,0.8182,1.0000,0.5000,0.8929
2,3,0.7500,0.8182,1.0000,0.6000,0.8333
3,4,1.0000,1.0000,1.0000,1.0000,1.0000
4,5,0.7500,0.8182,0.7500,0.7500,0.8750
5,6,0.7500,0.8182,1.0000,0.6000,0.9333
6,7,1.0000,1.0000,1.0000,1.0000,1.0000
7,8,0.5000,0.8182,0.5000,0.5000,0.8333
8,9,0.8889,0.9091,1.0000,0.8000,0.9833
9,10,0.3333,0.6364,0.3333,0.3333,0.6250


## Ranking of Important Genes according to L2 Regularised Logistic Regression

In [27]:
weightvals = (np.std(X_train,0)*logit_l2.coef_)
absweightvals = abs(weightvals)

In [28]:
featurecols_final = data.iloc[:,1:].columns

In [29]:
featurecols_final 

Index(['PHOX2B', 'LINC00682', 'TMEM33', 'DCAF4L1', 'SLC30A9', 'BEND4',
       'SHISA3', 'ATP8A1', 'GRXCR1', 'RN7SKP82',
       ...
       'SMIM9', 'SNORA36A', 'SNORA56', 'TMLHE', 'VBP1',
       'IL9R|ENSG00000124334.12', 'SPRY3|ENSG00000168939.6',
       'VAMP7|ENSG00000124333.10', 'WASH6P|ENSG00000182484.10',
       'WASIR1|ENSG00000185203.7'],
      dtype='object', length=987)

In [22]:
featureweightseries = pd.Series(data=absweightvals[0],index=featurecols_final)
featureweightseries.sort_values(ascending=False, inplace=True)
featureweightseries

COMMD8     0.179743
CD4        0.138121
CXorf64    0.128229
RAB21      0.080626
LGR5       0.080626
             ...   
GPR4       0.000296
OPA3       0.000296
VASP       0.000296
PPM1N      0.000296
ZNF222     0.000296
Length: 987, dtype: float64